# IE402 Assignment 2
### By: 201801015
### 11-10-2020

In [32]:
import numpy as np  
from fractions import Fraction 

In [61]:
def simplex():
    # inputs
    n, m = input("Enter number of basis variables and number of constraints seperated by a space:").split(" ")
    n, m = int(n), int(m)
    #temp1 = np.empty([n+m], dtype=float)
    
    # A will contain the coefficients of the constraints 
    A = np.array([[3, 2, 1, 1, 0], [2, 1, 2, 0, 1]]) 
    # b will contain the amount of resources  
    b = np.array([3, 2])            
    # c will contain coefficients of objective function Z       
    c = np.array([1, 1, 3, 0, 0])              
    Cj = np.array(np.append(["" ,"", "Cj"],c))
    # B will contain the basic variables that make identity matrix 
    cb = np.array(c[4]) 
    B = np.array([[3], [4]])           
     # cb contains their corresponding coefficients in Z    
    cb = np.vstack((cb, c[3]))         
    xb = np.transpose([b])
    MR = np.array([[0], [0]])  
    OP = np.array([[0], [0]])  
    # combine matrices B and cb 
    table = np.hstack((B, cb))              
    table = np.hstack((table, xb))          
    # combine matrices B, cb and xb 
    # finally combine matrix A to form the complete simplex table 
    table = np.hstack((table, A))        
    table = np.hstack((table, MR))  
    table = np.hstack((table, OP)) 
    # change the type of table to float 
    table = np.array(table, dtype ='float')  
    # inputs end 

    # if min problem, make this var 1 
    MIN = 0

    #print("Table at itr = 0") 
    #print("B \tCB \tXB \ty1 \ty2 \ty3 \ty4 \ty5 \tMR") 
    #table[:, 0] = table[:, 0] + 1
    #for row in table: 
    #    for el in range(len(row)-1):
    #        # limit the denominator under 100 
    #        print(Fraction(str(row[el])).limit_denominator(100), end ='\t')  
    #    print() 
    #print() 
    #table[:, 0] = table[:, 0] - 1
    print("\n\nSimplex Algorithm: Tabular Method\n\n") 

    # when optimality reached it will be made 1 
    reached = 0     
    itr = 1
    unbounded = 0
    alternate = 0

    while reached == 0: 

        # calculate Relative profits-> -cj + zj for non-basics 
        i = 0
        rel_prof = [] 
        while i<len(A[0]): 
            rel_prof.append(-c[i] + np.sum(table[:, 1]*table[:, 3 + i])) 
            i = i + 1

        flag = 0
        for profit in rel_prof: 
            if profit<0: 
                flag = 1
                break
            # if all relative profits <= 0 
        if flag == 0: 
            reached = 1

        # kth var will enter the basis 
        newIn = 99999
        for profit in rel_prof: 
            if profit<newIn:
                newIn = profit
        k = rel_prof.index(newIn) 
        minn = 99999
        i = 0; 
        r = -1

        # min ratio test (only positive values) 
        while i<len(table): 
            if (table[:, 2][i]>0 and table[:, 3 + k][i]>0):  
                val = table[:, 2][i]/table[:, 3 + k][i] 
                table[:,3+len(c)][i] = val
                if val<minn: 
                    minn = val 
                    r = i     # leaving variable 
            i+= 1

        print("Iteration: ", end =' ') 
        print(itr) 
        for ele in Cj:
            print(ele, end="\t")
        print()
        print("XB \tCB \tB \ta1 \ta2 \ta3 \ta4 \ta5 \tMR \tO/P") 
        table[:, 0] = table[:, 0] + 1
        for row in table: 
            if reached == 1:
                for el in range(len(row)-2):
                    print(Fraction(str(row[el])).limit_denominator(100), end ='\t')  
                print() 
            else:
                for el in range(len(row)):
                    print(Fraction(str(row[el])).limit_denominator(100), end ='\t')  
                print() 
        table[:, 0] = table[:, 0] - 1

        print("     Zj - Cj ", end ='\t\t') 
        for profit in rel_prof: 
            print(Fraction(str(profit)).limit_denominator(100), end ='\t') 
        print() 
        if reached == 0:
            pivot = table[r][3 + k] 
            print("pivot element: ", end =" ") 
            print(Fraction(pivot+1).limit_denominator(100)) 
        print() 
        
        i = 0
        b_var = table[:, 0] 
        
        # checking for alternate solution 
        while i<len(A[0]): 
            j = 0
            present = 0
            while j<len(b_var): 
                if int(b_var[j]) == i: 
                    present = 1
                    break; 
                j+= 1
            if present == 0: 
                if rel_prof[i] == 0: 
                    alternate = 1
                    print("Alternate Solutions") 
                    # print(i, end =" ") 
            i+= 1
            
        # if no min ratio test was performed 
        if r ==-1: 
            unbounded = 1
            print("Unbounded") 
            break

        #print("pivot element index:", end =' ') 
        #print(np.array([r, 3 + k])) 
        if flag == 0: 
            print("The optimum solution is:") 
            break

        # perform row operations 
        # divide the pivot row with the pivot element 
        table[r, 2:len(table[0])] = table[ 
                r, 2:len(table[0])] / pivot 

        # do row operation on other rows 
        i = 0
        while i<len(table): 
            if i != r: 
                table[i, 2:len(table[0])] = table[i, 2:len(table[0])] - table[i][3 + k] * table[r, 2:len(table[0])] 
            i += 1


        # assign the new basic variable 
        table[r][0] = k 
        table[r][1] = c[k] 

        #print() 
        itr+= 1


    #print()

    if unbounded == 1: 
        print("UNBOUNDED LPP") 
        exit() 
    if alternate == 1: 
        print("ALTERNATE Solution") 
    
    basis = [] 
    i = 0
    sum = 0
    while i<len(table): 
        sum += c[int(table[i][0])]*table[i][2] 
        temp = "x"+str(int(table[i][0])+1) 
        basis.append(temp) 
        i+= 1
    #print 0 to n-1 basis variables
    for ele in range(n): 
        if ele in table[:,0]:
            print("x"+str(ele+1)+" = "+str(table[:,2][np.where(table[:,0] == ele)][0]), end=", ")
        else:
            print("x"+str(ele+1)+" = 0", end=", ")
    print("and Z =", end=" ")
    # if MIN problem make z negative 
    if MIN == 1: 
        print(-Fraction(str(sum)).limit_denominator(100)) 
    else: 
        print(Fraction(str(sum)).limit_denominator(100)) 
    
    #print("Final Basis: ", end =" ") 
    #print(basis) 
    
    print()
    print("Simplex Finished Successfully!") 
    
# call simplex() function
simplex()

Enter number of basis variables and number of constraints seperated by a space:3 2


Simplex Algorithm: Tabular Method


Iteration:  1
		Cj	1	1	3	0	0	
XB 	CB 	B 	a1 	a2 	a3 	a4 	a5 	MR 	O/P
4	0	3	3	2	1	1	0	3	0	
5	0	2	2	1	2	0	1	1	0	
     Zj - Cj 		-1	-1	-3	0	0	
pivot element:  3

Iteration:  2
		Cj	1	1	3	0	0	
XB 	CB 	B 	a1 	a2 	a3 	a4 	a5 	MR 	O/P
4	0	2	2	3/2	0	1	-1/2	
3	3	1	1	1/2	1	0	1/2	
     Zj - Cj 		2	1/2	0	0	3/2	

The optimum solution is:
x1 = 0, x2 = 0, x3 = 1.0, and Z = 3

Simplex Finished Successfully!
